In [75]:
# Importamos las librerías necesarias
import numpy as np
import pandas as pd
import seaborn as snb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
# from sklearn.impute import SimpleImputer
# imp = SimpleImputer(strategy='mean')

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_log_error

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline
pd.options.mode.chained_assignment = None

In [76]:
# Jalamos los datos del dataset
data_train = pd.read_csv('datasets/train.csv', encoding='utf-8', sep=',')
data_test = pd.read_csv('datasets/test.csv', encoding='utf-8', sep=',')
dataf = pd.concat((data_train, data_test)).reset_index(drop=True)

In [78]:
# Obtenemos inforación del dataset
dataf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131588 entries, 0 to 131587
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    131588 non-null  object 
 1   host_1                131588 non-null  int64  
 2   host_2                131588 non-null  object 
 3   host_3                131588 non-null  object 
 4   host_4                131588 non-null  object 
 5   host_5                131588 non-null  object 
 6   host_6                131588 non-null  object 
 7   host_7                131588 non-null  object 
 8   host_8                131588 non-null  int64  
 9   property_1            131588 non-null  float64
 10  property_2            131588 non-null  float64
 11  property_3            131588 non-null  float64
 12  property_4            131588 non-null  object 
 13  property_5            131588 non-null  object 
 14  property_6            131588 non-null  int64  
 15  

In [79]:
# Obtenemos el número de registros y columnas del dataset
dataf.shape

(131588, 33)

In [80]:
# Imprimimos en pantalla los 5 primeros registros
dataf.head(5)

,id,host_1,host_2,host_3,host_4,host_5,host_6,host_7,host_8,property_1,...,property_15,property_16,property_max_booking,property_min_booking,property_price,reviews_each_month,reviews_last_30_days,reviews_total,total_listings,max_rating_class
0,d23eea91dda3cfc8,5,t,t,f,within an hour,99%,100%,111,3.0,...,t,38,1125,2,$792.00,1.43,0,36,116.0,location
1,91aa7023cbfe98f8,6,t,t,f,within a day,80%,0%,8,9.0,...,f,36,1124,31,$166.00,0.27,0,29,7.0,cleanliness
2,ad155140fab30327,6,t,t,t,within an hour,100%,100%,2,6.0,...,t,45,1125,1,$178.00,4.09,2,24,1.0,checkin
3,f90f94bf3b78a43e,6,t,t,f,within a day,80%,28%,1,6.0,...,f,26,1125,3,$61.00,0.29,1,5,1.0,accuracy
4,cb32a50ba2199d4c,1,t,t,f,within an hour,100%,100%,1,0.0,...,f,73,8,2,$340.00,1.32,0,8,0.0,rating


In [6]:
# Creamos una función que devuleva un dataframe con el conteo y porcentaje de nulos
def porcentaje_nulos(col=''):
  null_counts = pd.DataFrame()
  null_counts['conteo'] = dataf.isnull().sum()
  null_counts = null_counts[null_counts['conteo']>0].reset_index()
  null_counts['porcentaje'] = (null_counts['conteo']/dataf.shape[0] * 100).round(2)
  if col in ['','all']:
    return null_counts
  return null_counts['porcentaje'].loc[null_counts['index'] == col][0]


In [7]:
porcentaje_nulos()

,index,conteo,porcentaje
0,max_rating_class,46056,35.0


In [81]:
# columnas para transformar true or false a 1 y 0 respectivamente
columas_true_false = ['host_2','host_3','host_4']
def recodificar_campos_true_false(lista_columna):
  for columna in lista_columna:
    dataf[columna] = dataf[columna].apply(lambda x : 1 if x =='t' else 0)
  return dataf

recodificar_campos_true_false(columas_true_false)[['host_2','host_3','host_4']].head(5)

,host_2,host_3,host_4
0,1,1,0
1,1,1,0
2,1,1,1
3,1,1,0
4,1,1,0


In [82]:
# Clasificando y recodificando la  ['host_5']
  # wah = within an hour
  # wad = within a day 
  # wafh = within a few hours
  # afdom = a few days or more

def recodificar_host_5(valor):
    
    if valor == 'within an hour':
      return 'wah'
    elif valor == 'within a day':
      return 'wad'
    elif valor == 'within a few hours':
      return 'wafh'
    elif valor == 'a few days or more':
      return 'afdom'
    else:
      return 'na'
    
dataf['host_5'] = dataf['host_5'].apply(lambda x : recodificar_host_5(x) if recodificar_host_5(x) != 'na' else x)
dataf['host_5'].head(5)

0    wah
1    wad
2    wah
3    wad
4    wah
Name: host_5, dtype: object

In [83]:
# columnas para quitar el símbolo de porcentaje(%)
columa_quitar_simbolo_porcentaje = ['host_6','host_7']
def recodificar_campos_quitar_simbolo_porcentaje(lista_columna):
  for columna in lista_columna:
    dataf[columna] = dataf[columna].apply(lambda x: x.replace('%',''))
  return dataf

recodificar_campos_quitar_simbolo_porcentaje(columa_quitar_simbolo_porcentaje)[['host_6','host_7']].head(5)

,host_6,host_7
0,99,100
1,80,0
2,100,100
3,80,28
4,100,100


In [84]:
# Clasificando y recodificando la  ['property_5']
  # eha = Entire home/apt
  # pr = Private room
  # hr = Hotel room
  # sr = Shared room

def recodificar_property_5(valor):
    
    if valor == 'Entire home/apt':
      return 'eha'
    elif valor == 'Private room':
      return 'pr'
    elif valor == 'Hotel room':
      return 'hr'
    elif valor == 'Shared room':
      return 'sr'
    else:
      return 'na'
    
dataf['property_5'] = dataf['property_5'].apply(lambda x : recodificar_property_5(x) if recodificar_property_5(x) != 'na' else x)
dataf['property_5'].head(5)

0    eha
1    eha
2     pr
3    eha
4    eha
Name: property_5, dtype: object

In [73]:
# ['accuracy' 'checkin' 'cleanliness' 'communication' 'location' 'rating' 'value']
def recodificarmax_rating_class(valor):
    if valor == 'accuracy':
      return 0
    elif valor == 'checkin':
      return 1
    elif valor == 'cleanliness':
      return 2
    elif valor == 'communication':
      return 3
    elif valor == 'location':
      return 4
    elif valor == 'rating':
      return 5
    elif valor == 'value':
      return 6
    else:
      return 'na'
    
data_train['max_rating_class'] = data_train['max_rating_class'].apply(lambda x : recodificarmax_rating_class(x) if recodificarmax_rating_class(x) != 'na' else x)
data_train['max_rating_class'].head(5)

0    4
1    2
2    1
3    0
4    5
Name: max_rating_class, dtype: int64

In [85]:
list(dataf.property_4.unique())

['Entire rental unit',
 'Room in boutique hotel',
 'Entire condominium (condo)',
 'Entire residential home',
 'Entire serviced apartment',
 'Entire townhouse',
 'Entire loft',
 'Private room in residential home',
 'Private room in townhouse',
 'Entire cottage',
 'Private room in rental unit',
 'Room in serviced apartment',
 'Private room in bed and breakfast',
 'Entire guesthouse',
 'Entire guest suite',
 'Entire bungalow',
 'Private room in condominium (condo)',
 'Private room in villa',
 'Private room in serviced apartment',
 'Entire cabin',
 'Private room in kezhan',
 'Room in hotel',
 'Entire villa',
 'Entire bed and breakfast',
 'Room in hostel',
 'Room in bed and breakfast',
 'Shared room in residential home',
 'Tiny house',
 'Private room in hostel',
 'Private room in tiny house',
 'Private room in resort',
 'Private room in guest suite',
 'Shared room in boat',
 'Private room in loft',
 'Shared room in rental unit',
 'Castle',
 'Room in aparthotel',
 'Tower',
 'Private room in 

In [13]:
len(dataf.property_4.unique())

135

In [86]:
dataf.loc[:,dataf.columns[:15]]

,id,host_1,host_2,host_3,host_4,host_5,host_6,host_7,host_8,property_1,property_2,property_3,property_4,property_5,property_6
0,d23eea91dda3cfc8,5,1,1,0,wah,99,100,111,3.0,25.988190,-80.118700,Entire rental unit,eha,8
1,91aa7023cbfe98f8,6,1,1,0,wad,80,0,8,9.0,45.518120,-73.590680,Entire rental unit,eha,6
2,ad155140fab30327,6,1,1,1,wah,100,100,2,6.0,52.366010,4.880780,Room in boutique hotel,pr,2
3,f90f94bf3b78a43e,6,1,1,0,wad,80,28,1,6.0,44.802970,-0.542060,Entire condominium (condo),eha,4
4,cb32a50ba2199d4c,1,1,1,0,wah,100,100,1,0.0,30.379070,-97.924670,Entire residential home,eha,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131583,c1db5636e69d106d,7,1,1,0,wah,100,100,232,6.0,20.710001,-156.444049,Entire condominium (condo),eha,6
131584,6e304e03deda1b65,5,1,1,0,wad,60,44,1,1.0,33.919610,-118.337680,Private room in rental unit,pr,4
131585,49ddce98af5d334b,6,1,1,0,wah,100,100,3,2.0,46.961470,-65.596790,Entire rental unit,eha,3
131586,6f181667336285eb,5,1,1,0,wad,100,100,1,8.0,41.405210,2.179980,Entire rental unit,eha,4


In [15]:
dataf.loc[:,dataf.columns[:15]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131588 entries, 0 to 131587
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          131588 non-null  object 
 1   host_1      131588 non-null  int64  
 2   host_2      131588 non-null  int64  
 3   host_3      131588 non-null  int64  
 4   host_4      131588 non-null  int64  
 5   host_5      131588 non-null  object 
 6   host_6      131588 non-null  object 
 7   host_7      131588 non-null  object 
 8   host_8      131588 non-null  int64  
 9   property_1  131588 non-null  float64
 10  property_2  131588 non-null  float64
 11  property_3  131588 non-null  float64
 12  property_4  131588 non-null  object 
 13  property_5  131588 non-null  object 
 14  property_6  131588 non-null  int64  
dtypes: float64(3), int64(6), object(6)
memory usage: 15.1+ MB


In [87]:
# Cambiando tipo de columna por 'int64'
columnas = ['host_6', 'host_7',	'host_8']
for col in columnas:
  dataf[col] = dataf[col].astype('int64')

In [88]:
# Cambiando nombre a columnas ['property_2','property_3'] por ['lat','lon'] respectivamente
dataf.rename(columns={'property_2':'lat', 'property_3':'lon'}, inplace=True)

# ROBERT

In [89]:
# Separacion y conversion de datos 
new1 = dataf["property_7"].str.split(" ", n = 1, expand = True) 
dataf["property_7"] = new1[0]
dataf["property_7_bano"] = new1[1]
dataf['property_7'] = pd.to_numeric(dataf.property_7, errors='coerce')
dataf["property_10"] = dataf["property_10"].replace("t",1)
dataf["property_10"] = dataf["property_10"].replace("f",0)
dataf["property_15"] = dataf["property_15"].replace("t",1)
dataf["property_15"] = dataf["property_15"].replace("f",0)
new2 = dataf["property_price"].str.split("$", n = 1, expand = True) 
dataf['property_price']= new2[1]
dataf['property_price'] = pd.to_numeric(dataf.property_price, errors='coerce')

In [92]:
# Seperamos las columnas de tipo Objeto y numérico
col_objets = []
for i in dataf.columns:
    if dataf[i].dtype == object:
        col_objets.append(i)

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
col_numerics = []
for i in dataf.columns:
    if dataf[i].dtype in numeric_dtypes:
        col_numerics.append(i)

In [93]:
print(f"El número de columas de tipo objeto son: {len(col_objets)}")
print(f"El número de columas de tipo numérico son: {len(col_numerics)}")

El número de columas de tipo objeto son: 6
El número de columas de tipo numérico son: 28


In [94]:
df_final_numerics = dataf[col_numerics]

In [97]:
col_objets

['id',
 'host_5',
 'property_4',
 'property_5',
 'max_rating_class',
 'property_7_bano']

In [95]:
# Definimos una función que me permite aplicar Onehotencoder
def category_onehotencoder(cols, df_encoder):
    onehot_encoder = OneHotEncoder(sparse = False)
    df_ = df_final_numerics.copy()
    for col in cols:
        col_encoder = df_encoder[col].values.reshape(-1,1)
        onehot_encoder.fit(col_encoder)
        cols_encoded = onehot_encoder.transform(col_encoder)

        for i, e in enumerate(onehot_encoder.categories_[0]):
            df_[e+"_encode"] = cols_encoded[:,i]
    return df_

In [100]:
col_objets = ['host_5', 'property_4', 'property_5', 'property_7_bano']

In [103]:
dataf[col_objets].isnull().sum()

host_5              0
property_4          0
property_5          0
property_7_bano    80
dtype: int64

In [101]:
# Asignamos a la variable 'df_final_encode' el resultado del Onehotencoder
df_final_encode = category_onehotencoder(col_objets,dataf)

/var/folders/rz/0d_g_df14hn4j_5kyrct5lg00000gn/T/ipykernel_76220/1499502634.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_[e+"_encode"] = cols_encoded[:,i]
/var/folders/rz/0d_g_df14hn4j_5kyrct5lg00000gn/T/ipykernel_76220/1499502634.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_[e+"_encode"] = cols_encoded[:,i]
/var/folders/rz/0d_g_df14hn4j_5kyrct5lg00000gn/T/ipykernel_76220/1499502634.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, w

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

## Usando el modelo "Extreme Gradient Boosting"

In [70]:
# Importamos la librería
import xgboost as xgb

In [71]:
data_train.iloc[:,-1].head(5)

0       location
1    cleanliness
2        checkin
3       accuracy
4         rating
Name: max_rating_class, dtype: object

In [74]:
# Creamos el objeto XGBRegressor y hacemos el split
modelo_xgbc = xgb.XGBClassifier(enable_categorical=True)

X_df_train = data_train.iloc[:,1:-1]
y_df_train = data_train.iloc[:,-1]

X_train_xgbc, X_test_xgbc, y_train_xgbc, y_test_xgbc = train_test_split(X_df_train, y_df_train, test_size=0.33, random_state=42)

# Entrenamos el modelo XGBRegressor
modelo_xgbc.fit(X_train_xgbc, y_train_xgbc)

# Predecimos sobre nuestro set de entrenamieto
# y_train_pred_xgbc = modelo_xgbc.predict(X_train_xgbc)

# Predecimos sobre nuestro set de test
# y_test_pred_xgbc = modelo_xgbc.predict(X_test_xgbc)


ValueError: Experimental support for categorical data is not implemented for current tree method yet.

In [67]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85532 entries, 0 to 85531
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    85532 non-null  object 
 1   host_1                85532 non-null  int64  
 2   host_2                85532 non-null  object 
 3   host_3                85532 non-null  object 
 4   host_4                85532 non-null  object 
 5   host_5                85532 non-null  object 
 6   host_6                85532 non-null  object 
 7   host_7                85532 non-null  object 
 8   host_8                85532 non-null  int64  
 9   property_1            85532 non-null  float64
 10  property_2            85532 non-null  float64
 11  property_3            85532 non-null  float64
 12  property_4            85532 non-null  object 
 13  property_5            85532 non-null  object 
 14  property_6            85532 non-null  int64  
 15  property_7         